# Ensemble Learning
In statistics and machine learning, **ensemble methods** use multiple learning algorithms to obtain better predictive performance than could be obtained from any of the constituent learning algorithms alone. Simply put,

1. Ensemble models in machine learning combine the decisions from multiple models to improve the overall performance.


2. The main causes of error in learning models are due to noise, bias and variance. Ensemble methods help to minimize these factors.


3. Ensembles are a divide and conquer approach used to improve performance.

Ensemble learning can help you to build a really robust model from a few "weak" models, which eliminates a lot of the model tuning that would else be needed to achieve good results. They are especially popular in data science competitions because for these competitions the highest accuracy is more important than the runtime or interpretability of the model.

In [13]:
import pathlib
import numpy as np
import pandas as pd
# import xgboost as xgb
# import lightgbm as lgb
from statistics import mode
from scipy.stats import norm, skew
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingClassifier, AdaBoostRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.base import BaseEstimator, TransformerMixin, clone, RegressorMixin
from sklearn.metrics import f1_score

# House Prices

In this notebook, we will go through the most common ensembling techniques out there. We will work on the [House Prices Regression dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview) which can be freely downloaded from Kaggle.

In [2]:
path = pathlib.Path("datasets")

In [3]:
# Load in the train and test dataset
train = pd.read_csv(path/'train.csv')
test = pd.read_csv(path/'test.csv')

train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
train.isnull().sum().sort_values(ascending=False)

PoolQC           1453
MiscFeature      1406
Alley            1369
Fence            1179
FireplaceQu       690
LotFrontage       259
GarageCond         81
GarageType         81
GarageYrBlt        81
GarageFinish       81
GarageQual         81
BsmtExposure       38
BsmtFinType2       38
BsmtFinType1       37
BsmtCond           37
BsmtQual           37
MasVnrArea          8
MasVnrType          8
Electrical          1
Utilities           0
YearRemodAdd        0
MSSubClass          0
Foundation          0
ExterCond           0
ExterQual           0
Exterior2nd         0
Exterior1st         0
RoofMatl            0
RoofStyle           0
YearBuilt           0
                 ... 
GarageArea          0
PavedDrive          0
WoodDeckSF          0
OpenPorchSF         0
3SsnPorch           0
BsmtUnfSF           0
ScreenPorch         0
PoolArea            0
MiscVal             0
MoSold              0
YrSold              0
SaleType            0
Functional          0
TotRmsAbvGrd        0
KitchenQua

In [5]:
# Save the 'Id' column
train_ID = train['Id']
test_ID = test['Id']

# Drop id column because is is unnecessary for making predictions
train.drop('Id', axis=1, inplace=True)
test.drop('Id', axis=1, inplace=True)

## Feature Engineering
Feature engineering is important for traditional machine learning, but it's not the focus of this project. You can take reference of this [Stacked Regressions to Predict House Prices](https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard) for data exploration on this dataset.

In [6]:
train['SalePrice'] = np.log1p(train['SalePrice'])

In [7]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print('all_data size is: {}'.format(all_data.shape))

all_data size is: (2919, 79)


C:\Users\z00045494\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


### Missing Data

In [8]:
all_data_na = (all_data.isnull().sum() / len(all_data))*100
all_data_na = all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ration': all_data_na})
missing_data.head(20)

,Missing Ration
PoolQC,99.657417
MiscFeature,96.402878
Alley,93.216855
Fence,80.438506
FireplaceQu,48.646797
LotFrontage,16.649538
GarageQual,5.447071
GarageCond,5.447071
GarageFinish,5.447071
GarageYrBlt,5.447071


### Impute missing values

In [9]:
all_data['PoolQC'] = all_data['PoolQC'].fillna('None')
all_data['MiscFeature'] = all_data['MiscFeature'].fillna('None')
all_data['Alley'] = all_data['Alley'].fillna('None')
all_data['Fence'] = all_data['Fence'].fillna('None')
all_data['FireplaceQu'] = all_data['FireplaceQu'].fillna('None')
all_data['LotFrontage'] = all_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
all_data['MasVnrType'] = all_data['MasVnrType'].fillna('None')
all_data['MasVnrArea'] = all_data['MasVnrArea'].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data = all_data.drop(['Utilities'], axis=1)
all_data['Functional'] = all_data['Functional'].fillna('Typ')
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna('None')

In [10]:
all_data_na = (all_data.isnull().sum() / len(all_data))*100
all_data_na = all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio':all_data_na})
missing_data.head()

,Missing Ratio


In [11]:
train = all_data[:ntrain]
test = all_data[ntrain:]
train.to_csv(path/'train_cleaned.csv')
test.to_csv(path/'test_cleaned.csv')

### Transform Numerical Features to Categorical

In [12]:
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)

all_data['OverallCond'] = all_data['OverallCond'].astype(str)

all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

In [13]:
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

for c in cols:
    lbl = LabelEncoder()
    lbl.fit(list(all_data[c].values))
    all_data[c] = lbl.transform(list(all_data[c].values))
    
print('Shape all_data {}'.format(all_data.shape))

Shape all_data (2919, 78)


In [14]:
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

In [15]:
numeric_feats = all_data.dtypes[all_data.dtypes!='object'].index

# Check the skew of all the numeric features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print('Skew in numeric features:')
skewness = pd.DataFrame({'Skew': skewed_feats})
skewness.head(10)

Skew in numeric features:


,Skew
MiscVal,21.947195
PoolArea,16.898328
LotArea,12.822431
LowQualFinSF,12.088761
3SsnPorch,11.376065
LandSlope,4.975157
KitchenAbvGr,4.302254
BsmtFinSF2,4.146143
EnclosedPorch,4.003891
ScreenPorch,3.946694


In [16]:
skewness = skewness[abs(skewness) > 0.75]
print('There are {} skewed numerical features to Box Cox transform'.format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    all_data[feat] = boxcox1p(all_data[feat], lam)

There are 59 skewed numerical features to Box Cox transform


In [17]:
print(all_data.shape)
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(2919, 79)
(2919, 221)


In [18]:
train = all_data[:ntrain]
test = all_data[ntrain:]

train.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleCondition_Partial,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,11.692623,11.686189,0.0,0.730463,1.540963,1.820334,1.540963,11.170327,0.0,1.194318,...,0,0,0,0,0,0,0,0,0,1
1,12.792276,0.000000,0.0,0.730463,1.540963,1.820334,0.730463,12.062832,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,1
2,11.892039,11.724598,0.0,0.730463,1.540963,1.820334,1.194318,10.200343,0.0,1.194318,...,0,0,0,0,0,0,0,0,0,1
3,12.013683,11.354094,0.0,0.730463,1.540963,0.730463,1.540963,8.274266,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,1
4,12.510588,12.271365,0.0,0.730463,1.820334,1.820334,0.000000,10.971129,0.0,1.194318,...,0,0,0,0,0,0,0,0,0,1


In [19]:
train.dtypes.unique()

array([dtype('float64'), dtype('uint8')], dtype=object)

In [20]:
y_train

array([12.24769912, 12.10901644, 12.31717117, ..., 12.49313327,
       11.86446927, 11.90159023])

In [21]:
X = train.values

## Accuracy Function

In [22]:
n_folds = 5

def get_cv_scores(model, X, y, print_scores=True):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X)
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv = kf))
    if print_scores:
        print(f'Root mean squared error: {rmse.mean():.3f} ({rmse.std():.3f})')
    return [rmse]

## Base Models
Before we can start working through the different ensembling techniques we need to define some base models which will be used for ensembling. For this data-set, we will use a Lasso regression, Gradient Boosting, XGBoost, and lightGBM model.

We will validate the results using the root mean squared error and cross-validation.
### Lasso

In [23]:
lasso_model = make_pipeline(RobustScaler(), Lasso(alpha=0.0005, random_state=1))

In [24]:
%%time
get_cv_scores(lasso_model, X, y_train)

Root mean squared error: 0.124 (0.016)
Wall time: 628 ms


[array([0.1036135 , 0.13480558, 0.12784684, 0.10698948, 0.14677378])]

In [25]:
lasso_model.fit(X, y_train)
lasso_model.predict([X[0]])

array([12.23428681])

In [26]:
y_train[0]

12.24769911637256

In [27]:
lasso_model.score(X, y_train)

0.9285256686605036

### DecisionTreeRegressor

In [28]:
%%time
dt = DecisionTreeRegressor()
get_cv_scores(dt, X, y_train);

Root mean squared error: 0.210 (0.024)
Wall time: 284 ms


[array([0.18915863, 0.24631617, 0.201391  , 0.18288652, 0.23084938])]

### RandomForestRegressor

In [32]:
rf = RandomForestRegressor()

In [33]:
%%time
get_cv_scores(rf, X, y_train);

C:\Users\z00045494\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\z00045494\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\z00045494\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\z00045494\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.

Root mean squared error: 0.154 (0.009)
Wall time: 1.6 s


[array([0.14934295, 0.16510608, 0.14850745, 0.14352604, 0.16436697])]

### GradientBoostingRegressor

In [34]:
gbr = GradientBoostingRegressor()

In [35]:
%%time
get_cv_scores(gbr, X, y_train);

Root mean squared error: 0.125 (0.008)
Wall time: 5.03 s


[array([0.11448344, 0.13438681, 0.13498366, 0.1171377 , 0.12542057])]

In [36]:
gbr = GradientBoostingRegressor()
gbr.fit(X, y_train)
gbr.score(X, y_train)

0.9581780008371052

### XGBoost

In [ ]:
xgb_model = xgb.XGBRegressor()

In [ ]:
%%time
get_cv_scores(xgb_model, X, y_train);

### LightGBM

In [ ]:
lgb_model = lgb.LGBMRegressor()

In [ ]:
%%time
get_cv_scores(lgb_model, X, y_train);

## Simple Ensemble Techniques

In this section, we will look at a few simple but powerful techniques, namely:
1. Max Voting
2. Averaging
3. Weighted Averaging

### Max Voting

The **max voting** method is generally used for classification problems.

In this technique, multiple models are used to make predictions for each data point. The predictions by each model are considered as a "vote". The predictions which we get from the majority of the models are used as the final prediction.

For example, when you asked $5$ of your colleagues to rate your movie (out of $5$); we'll assume three of them rated it as $4$ while two of them gave it a $5$. Since the majority gave a rating of $4$, the final rating will be taken as $4$. You can consider this as taking the mode of all the predictions.

The result of max voting would be something like this:

| Colleague 1 | Colleague 2 | Colleague 3 | Colleague 4 | Colleague 5 |
| --- | --- | --- | --- | --- |
| 5 | 4 | 5 | 4 | 4 |

Because the House Prices dataset is for regression, we take `scikit-learn`'s Iris Dataset for example.

In [11]:
# Data preparation
iris = load_iris()
iris_X = iris.data
iris_y = iris.target

iris_X_train, iris_X_test, iris_y_train, iris_y_test = train_test_split(iris_X, iris_y, test_size=0.5)

# Models
iris_dt_model = DecisionTreeClassifier()
iris_knn_model = KNeighborsClassifier()
iris_lr_model = LogisticRegression()

# `fit()`
iris_dt_model.fit(iris_X_train, iris_y_train)
iris_knn_model.fit(iris_X_train, iris_y_train)
iris_lr_model.fit(iris_X_train, iris_y_train)

# `predict()`
iris_dt_pred = iris_dt_model.predict(iris_X_test)
iris_knn_pred = iris_knn_model.predict(iris_X_test)
iris_lr_pred = iris_lr_model.predict(iris_X_test)

# `f1_score()`
print("f1 Score of the decision tree model:", f1_score(iris_y_test, iris_dt_pred, average=None))
print("f1 Score of the KNN model:", f1_score(iris_y_test, iris_knn_pred, average=None))
print("f1 Score of the LR model:", f1_score(iris_y_test, iris_lr_pred, average=None))

# Max voting
iris_final_pred = np.array([])
for i in range(0, len(iris_X_test)):
    iris_final_pred = np.append(
        iris_final_pred, 
        mode([iris_dt_pred[i], iris_knn_pred[i], iris_lr_pred[i]])
    )
    
# F1 score of the ensemble model
print("f1 Score of the max voting model:", f1_score(iris_y_test, iris_final_pred, average=None))

f1 Score of the decision tree model: [1.         0.90909091 0.90909091]
f1 Score of the KNN model: [1.         0.89361702 0.87804878]
f1 Score of the LR model: [1.         0.97560976 0.9787234 ]
f1 Score of the max voting model: [1.         0.93333333 0.93023256]


C:\Users\z00045494\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\z00045494\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Alternatively, you can use `VotingClassifier` module in `sklearn`.

If the `voting` argument is set `hard`, the classifier uses predicted class labels for majority rule voting. If it's set `soft`, the classifier predicts the class label based on the argmax of the sums of the predicted probabilities, which is recommended for an ensemble of well-calibrated classifiers. 

In [19]:
voting_model = VotingClassifier(estimators=[('lr', iris_lr_model), ('knn', iris_knn_model), ('dt', iris_dt_model)], voting='hard')
voting_model.fit(iris_X_train, iris_y_train)
voting_pred = voting_model.predict(iris_X_test)
print("f1 Score of the ensemble model:", f1_score(iris_y_test, voting_pred, average=None))

f1 Score of the ensemble model: [1.         0.93333333 0.93023256]


C:\Users\z00045494\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\z00045494\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


### Averaging
**Averaging** is a simple method that is generally used for regression problems. Here the predictions are simply averaged to get a more robust result and even though this method is simple it almost always gives better results than a single model and therefore is always worth trying.

For ease of usage, we will create a class that inherits from `scikit-learn`'s `BaseEstimator`, `RegressorMixin`, and `TransformerMixin` classes because that way we only need to overwrite the fit and predict methods to get a working model which can be used like any other `scikit-learn` model.

In [ ]:
class AveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)
        return self
    
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

The model can now be used like any other `scikit-learn` model. The only difference is that when creating an object we need to pass the base models which will then be trained in the `fit` method and used for predictions in the `prediction` method.

By averaging three of our base models - Lasso regression, GBM, and XGBoost - we get a significant accuracy increase.

In [ ]:
%%time
averaged_model_1 = AveragedModels([gbr, lasso_model, xgb_model])
get_cv_scores(averaged_model_1, X, y_train);

In [ ]:
%%time
averaged_model_2 = AveragedModels([gbr, lasso_model, xgb_model, lgb_model])
get_cv_scores(averaged_model_2, X, y_train);

### Weighted Average
Averaging models is great and simple but it has one major flaw and that is that most of the time one model has more predictive power than another and therefore we want to give it more weight on the final predictions.
We can achieve this by passing a weight for each of the models and then multiplying the predictions of each model with the corresponding weight. The only thing we need to look out for is that the weights need to add up to $1$ in order not to change the scale of the predictions.

In [ ]:
class WeightedAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models, weights):
        self.models = models
        self.weights = weights
        assert sum(self.weights) == 1
        
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)
            
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.sum(predictions*self.weights, axis=1)

To now create the model we not only need to pass the base models but also an array of weights.

In [ ]:
%%time
weighted_average_model_1 = WeightedAveragedModels([gbr, lasso_model, xgb_model], [0.3, 0.3, 0.4])
get_cv_scores(weighted_average_model_1, X, y_train);

In [ ]:
%%time
weighted_average_model_2 = WeightedAveragedModels([gbr, lasso_model, xgb_model], [0.3, 0.45, 0.25])
get_cv_scores(weighted_average_model_2, X, y_train);

## Advanced Ensemble Techniques
Ensemble methods are meta-algorithms that combine several machine learning techniques into one predictive model in order to **decrease variance** (bagging), **bias** (boosting), or **improve predictions** (stacking).

Ensemble methods can be divided into two groups:

* **Sequential** ensemble methods where the base learners are generated sequentially (e.g. AdaBoost). The basic motivation of sequential methods is to exploit the dependence between the base learners. The overall performance can be boosted by weighing previously mislabeled examples with higher weight.


* **Parallel** ensemble methods where the base learners are generated in parallel (e.g. Random Forest). The basic motivation of parallel methods is to exploit independence between the base learners since the error can be reduced dramatically by averaging.

Most ensemble methods use a single base learning algorithm to produce homogeneous base learners, i.e. learners of the same type, leading to homogeneous ensembles.

There are also some methods that use heterogeneous learners, i.e. learners of different types, leading to heterogeneous ensembles. In order for ensemble methods to be more accurate than any of its individual members, the base learners have to be as accurate as possible and as diverse as possible.

### Bagging

**Bagging** stands for bootstrap aggregation. One way to reduce the variance of an estimate is to average together multiple estimates. For example, we can train M different trees on different subsets of the data (chosen randomly with replacement) and compute the ensemble:

$f(x) = 1/M \sum^{M}_{m=1} f_{m}(x)$

Bagging uses bootstrap sampling to obtain the data subsets for training the base learners. For aggregating the outputs of base learners, bagging uses voting for classification and averaging for regression.

![bagging-1](images/bagging-1.png)

1. Multiple subsets are created from the original dataset, selecting observations with replacement.

2. A base model (weak model) is created on each of these subsets.

3. The models run in parallel and are independent of each other.

4. The final predictions are determined by combining the predictions from all the models.

![bagging-2](images/bagging-2.png)

As each model is exposed to a different subset of data and we use their collective output at the end, so we are making sure that problem of overfitting is taken care of by not clinging too closely to our training data set. Thus, Bagging helps us to reduce the variance error.

Combinations of multiple models decreases variance, especially in the case of unstable models, and may produce a more reliable prediction than a single model.

In **random forests**, each tree in the ensemble is built from a sample drawn with replacement (i.e. a bootstrap sample) from the training set. In addition, instead of using all the features, a random subset of features is selected, further randomizing the tree.

As a result, the bias of the forest increases slightly, but due to the averaging of less correlated trees, its variance decreases, resulting in an overall better model.

In an **extremely randomized trees** algorithm randomness goes one step further: the splitting thresholds are randomized. Instead of looking for the most discriminative threshold, thresholds are drawn at random for each candidate feature and the best of these randomly-generated thresholds is picked as the splitting rule. This usually allows reduction of the variance of the model a bit more, at the expense of a slightly greater increase in bias.

In [ ]:
class BaggingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]

        for model in self.models_:
            X_tmp, y_tmp = self.subsample(X, y)
            model.fit(X_tmp, y_tmp)
        
        return self
    
    # Create a random subsample from the dataset with replacement
    def subsample(self, X, y, ratio=1.0):
        X_new, y_new = list(), list()
        n_sample = round(len(X) * ratio)
        while len(X_new) < n_sample:
            index = np.random.randint(len(X))
            X_new.append(X[index])
            y_new.append(y[index])
        return X_new, y_new
    
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

With this technique, we are able to create models which are highly uncorrelated and therefore perform really well when ensembled as can be seen when we compare a single decision tree and a random forest.

In [ ]:
%%time
bagging_model = BaggingModels([gbr, lasso_model, xgb_model])
get_cv_scores(bagging_model, X, y_train);

In [ ]:
%%time
bagging_model = BaggingModels([gbr, lasso_model, xgb_model]*2)
get_cv_scores(bagging_model, X, y_train);

### Boosting

**Boosting** refers to a family of algorithms that are able to convert weak learners to strong learners. The main principle of boosting is to fit a sequence of weak learners - models that are only slightly better than random guessing, such as small decision trees - to weighted versions of the data. More weight is given to examples that were misclassified by earlier rounds.

The predictions are then combined through a weighted majority vote (classification) or a weighted sum (regression) to produce the final prediction. The principal difference between boosting and the committee methods, such as bagging, is that base learners are trained in sequence on a weighted version of the data.

Boosting follows the following steps:
1. Creates a subset from the original dataset (Initially, all datapoints are weighted equally).

2. Creates and trains a base-model on the subset.

3. The base model is used to make predictions on the whole dataset.

![boosting-1](images/boosting-1.png)

4. Errors of the predictions are calculated.

5. Incorrectly predicted datapoints (datapoints with bigger error) are given higher weights.

6. Another model is created on the datapoints with high weights.

![boosting-2](images/boosting-2.png)

7. Steps are repeated as long as needed.

8. The final model is the weighted average of all models (better models get higher weights).

![boosting-3](images/boosting-3.webp)

Thus, the boosting algorithm combines a number of weak learners to form a strong learner. The individual models would not perform well on the entire dataset, but they work well for some part of the dataset. Thus, each model actually boosts the performance of the ensemble.

![boosting-4](images/boosting-4.png)

Boosting in general decreases the bias error and builds strong predictive models. Boosting has shown better predictive accuracy than bagging, but it also tends to over-fit the training data as well. Thus, parameter tuning becomes a crucial part of boosting algorithms to make them avoid overfitting.

One of the first popular boosting implementations is called **AdaBoost**. To create an AdaBoost model we can simply use the `AdaBoostRegressor` model from `scikit-learn`.

In [ ]:
adaboost_model = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=3), n_estimators=50)

### Stacking

**Stacking** is an ensemble learning technique that combines multiple classification or regression models via a meta-classifier or a meta-regressor. The base level models are trained based on a complete training set, then the meta-model is trained on the outputs of the base level model as features. 

Simply put, stacking is an ensemble learning technique that uses predictions from multiple models (for example, decision tree, KNN or SVM) to build a new model. This model is used for making predictions on the test set.

The base level often consists of different learning algorithms and therefore stacking ensembles are often heterogeneous.

Below is a step-wise explanation for a simple stacked ensemble:

1. The train set is split into $10$ parts.

![stacking-1](images/stacking-1.png)

2. A base model (suppose a decision tree) is fitted on $9$ parts and predictions are made for the 10th part. This is done for each part of the train set.

![stacking-2](images/stacking-2.png)

3. The base model (in this case, decision tree) is then fitted on the whole train dataset.

4. Using this model, predictions are made on the test set.

![stacking-3](images/stacking-3.png)

5. Steps $2$ to $4$ are repeated for another base model (say KNN) resulting in another set of predictions for the train set and test set.

![stacking-4](images/stacking-4.png)

6. The predictions from the train set are used as features to build a new model.

![stacking-5](images/stacking-5.png)

7. This model is used to make final predictions on the test prediction set.

Stacking was introduced by Wolpert in the paper [Stacked Generalization](https://www.researchgate.net/publication/222467943_Stacked_Generalization) in 1992. It is a method that uses $k$-fold for training base models which then make predictions on the left out fold. These so-called out of fold predictions are then used to train another model - the meta model - which can use the information produced by the base models to make final predictions.

To implement this functionality we need to first train each base model $k$ times ($k...$ Number of folds) and then use their predictions to train our meta model.

To even get better results we can not only use the predictions of all the base models for training the meta model but also the initial features. Because of the added model complexity which is caused when adding the input features we should make a boolean parameter to determine whether we want to use input features.

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5, use_features_in_secondary=False):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
        self.use_features_in_secondary = use_features_in_secondary
        
    def fit(self, X, y):
        """Fit all the models on the given dataset"""
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=42)
        
        # Train cloned base models and create out-of-fold predictions
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
        
        if self.use_features_in_secondary:
            self.meta_model_.fit(np.hstack((X, out_of_fold_predictions)), y)
        else:
            self.meta_model_.fit(out_of_fold_predictions, y)
            
        return self
    
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        if self.use_features_in_secondary:
            return self.meta_model_.predict(np.hstack((X, meta_features)))
        else:
            return self.meta_model_.predict(meta_features)
    
    def predict_proba(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        if self.use_features_in_secondary:
            return self.meta_model_.predict_proba(np.hstack((X, meta_features)))
        else:
            return self.meta_model_.predict_proba(meta_features)

In [ ]:
%%time
stacking_model1 = StackingAveragedModels([gbr, lgb_model, xgb_model], lasso_model)
get_cv_scores(stacking_model1, X, y_train);

In [ ]:
%%time
stacking_model2= StackingAveragedModels([gbr, lgb_model, xgb_model], lasso_model, use_features_in_secondary=True)
get_cv_scores(stacking_model2, X, y_train);

In [ ]:
%%time
stacking_model3 = StackingAveragedModels([gbr, lgb_model, xgb_model, lasso_model], lasso_model)
get_cv_scores(stacking_model3, X, y_train);

In [ ]:
%%time
stacking_model4 = StackingAveragedModels([gbr, lgb_model, xgb_model, lasso_model], lasso_model, use_features_in_secondary=True)
get_cv_scores(stacking_model4, X, y_train);